# Storing Financial Data Efficiently

*[Coding along with Python for Algorithmic Trading, Yves Hilpisch, O'Reilly, 1st edition November 2020, ISBN-13: 978-1492053354]*

### Storing DataFrame Objects

In [2]:
import pandas as pd
from sample_data import generate_sample_data

In [2]:
print(generate_sample_data(rows=5, cols=4)) # printing sample financial data set

                            No0         No1         No2         No3
2021-01-01 00:00:00  100.000000  100.000000  100.000000  100.000000
2021-01-01 00:01:00  100.081746  100.037085   99.880699   99.945621
2021-01-01 00:02:00  100.078259   99.967327   99.919765   99.830417
2021-01-01 00:03:00  100.129863   99.937452   99.769169   99.743751
2021-01-01 00:04:00  100.007850  100.075159   99.683242   99.705222


In [3]:
# creating sample data set for storing example
%time
data = generate_sample_data(rows=5e6, cols=10).round(4) # 5.000.000 rows, 10 cols

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 2.62 μs


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000000 entries, 2021-01-01 00:00:00 to 2030-07-05 05:19:00
Freq: min
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   No0     float64
 1   No1     float64
 2   No2     float64
 3   No3     float64
 4   No4     float64
 5   No5     float64
 6   No6     float64
 7   No7     float64
 8   No8     float64
 9   No9     float64
dtypes: float64(10)
memory usage: 419.6 MB


In [5]:
# opening HDFStore object on disc for writing
h5 = pd.HDFStore('../../assets/hdf_store/data.h5', 'w')

In [6]:
%time h5['data'] = data # writing data frame object to disc

CPU times: user 3.49 ms, sys: 72.5 ms, total: 76 ms
Wall time: 272 ms


In [7]:
h5 # printing out meta information of database file

<class 'pandas.io.pytables.HDFStore'>
File path: ../../assets/hdf_store/data.h5

In [8]:
ls -n ../../assets/hdf_store/data.*

-rw-r--r--  1 501  20  440007240 20 Sep 14:07 ../../assets/hdf_store/data.h5


In [9]:
h5.close() # closing database file

In [10]:
# reading data from the file based HDFStore object
h5 = pd.HDFStore('../../assets/hdf_store/data.h5', 'r')

In [11]:
%time data_copy = h5['data'] # reading from stored file

CPU times: user 113 ms, sys: 169 ms, total: 281 ms
Wall time: 505 ms


In [12]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000000 entries, 2021-01-01 00:00:00 to 2030-07-05 05:19:00
Freq: min
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   No0     float64
 1   No1     float64
 2   No2     float64
 3   No3     float64
 4   No4     float64
 5   No5     float64
 6   No6     float64
 7   No7     float64
 8   No8     float64
 9   No9     float64
dtypes: float64(10)
memory usage: 419.6 MB


In [13]:
h5.close()

In [14]:
rm ../../assets/hdf_store/data.h5 # deleting file

In [15]:
# alternative way of storing a HDFStore that allows to append new data to the table
# and for searching over the table
# pandas.DataFrame.to_hdf
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_hdf.html
%time data.to_hdf('../../assets/hdf_store/data_q.h5', key='data', mode='w', format='table')

CPU times: user 2.89 s, sys: 1.3 s, total: 4.19 s
Wall time: 4.84 s


In [16]:
ls -n ../../assets/hdf_store/data_q.*

-rw-r--r--  1 501  20  446953369 20 Sep 14:07 ../../assets/hdf_store/data_q.h5


In [17]:
%time data_copy = pd.read_hdf('../../assets/hdf_store/data_q.h5', 'data')

CPU times: user 71.3 ms, sys: 130 ms, total: 201 ms
Wall time: 274 ms


In [18]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000000 entries, 2021-01-01 00:00:00 to 2030-07-05 05:19:00
Freq: min
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   No0     float64
 1   No1     float64
 2   No2     float64
 3   No3     float64
 4   No4     float64
 5   No5     float64
 6   No6     float64
 7   No7     float64
 8   No8     float64
 9   No9     float64
dtypes: float64(10)
memory usage: 419.6 MB


In [19]:
# Using table_frame object of the PyTables package to work with stored data
# PyTables’ documentation @ http://www.pytables.org/
import tables as tb

In [20]:
h5 = tb.open_file('../../assets/hdf_store/data_q.h5', 'r')

In [21]:
h5

File(filename=../../assets/hdf_store/data_q.h5, title=np.str_(''), mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) np.str_('')
/data (Group) np.str_('')
/data/table (Table(np.int64(5000000),)) np.str_('')
  description := {
  "index": Int64Col(shape=(), dflt=np.int64(0), pos=0),
  "values_block_0": Float64Col(shape=(np.int64(10),), dflt=np.float64(0.0), pos=1)}
  byteorder := 'little'
  chunkshape := (np.int64(2978),)
  autoindex := True
  colindexes := {
    "index": Index(6, mediumshuffle, zlib(1)).is_csi=False}

In [22]:
h5.root.data.table[:3] # printing the first three rows of the table

array([(1609459200000000000, [100.    , 100.    , 100.    , 100.    , 100.    , 100.    , 100.    , 100.    , 100.    , 100.    ]),
       (1609459260000000000, [ 99.9155, 100.0313, 100.0368,  99.9547, 100.0065, 100.0616,  99.9445, 100.0627, 100.0879, 100.0015]),
       (1609459320000000000, [ 99.886 , 100.0803,  99.913 ,  99.9213, 100.0344, 100.1168,  99.9712, 100.0912, 100.1552, 100.1331])],
      dtype=[('index', '<i8'), ('values_block_0', '<f8', (10,))])

In [23]:
h5.close()

In [24]:
rm ../../assets/hdf_store/data_q.h5 # deleting file

### Using TsTables

In [25]:
%%time
data = generate_sample_data(rows=2.5e6, cols=5, freq='1s').round(4)

CPU times: user 317 ms, sys: 66.3 ms, total: 383 ms
Wall time: 382 ms


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2500000 entries, 2021-01-01 00:00:00 to 2021-01-29 22:26:39
Freq: s
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   No0     float64
 1   No1     float64
 2   No2     float64
 3   No3     float64
 4   No4     float64
dtypes: float64(5)
memory usage: 114.4 MB


In [27]:
# poetry add git+https://github.com/yhilpisch/tstables.git
import tstables # importing pytables

In [28]:
import tables as tb # importing tables like provided by the author

In [29]:
class desc(tb.IsDescription):
    ''' Description of TsTables table structure.
    '''
    timestamp = tb.Int64Col(pos=0) # first column is timestamp represented as an int
    No0 = tb.Float64Col(pos=1) # all other columns contain floats
    No1 = tb.Float64Col(pos=2)
    No2 = tb.Float64Col(pos=3)
    No3 = tb.Float64Col(pos=4)
    No4 = tb.Float64Col(pos=5)

In [30]:
h5 = tb.open_file('../../assets/hdf_store/data_ts.h5ts', 'w') # opening new database file for writing

In [31]:
ts = h5.create_ts('/', 'data', desc) # tstable created with data and class based description

In [32]:
h5 # inspecting the data file

File(filename=../../assets/hdf_store/data_ts.h5ts, title=np.str_(''), mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) np.str_('')
/data (Group/Timeseries) np.str_('')
/data/y2024 (Group) np.str_('')
/data/y2024/m09 (Group) np.str_('')
/data/y2024/m09/d20 (Group) np.str_('')
/data/y2024/m09/d20/ts_data (Table(np.int64(0),)) np.str_('')
  description := {
  "timestamp": Int64Col(shape=(), dflt=np.int64(0), pos=0),
  "No0": Float64Col(shape=(), dflt=np.float64(0.0), pos=1),
  "No1": Float64Col(shape=(), dflt=np.float64(0.0), pos=2),
  "No2": Float64Col(shape=(), dflt=np.float64(0.0), pos=3),
  "No3": Float64Col(shape=(), dflt=np.float64(0.0), pos=4),
  "No4": Float64Col(shape=(), dflt=np.float64(0.0), pos=5)}
  byteorder := 'little'
  chunkshape := (np.int64(1365),)

In [33]:
%time ts.append(data) # appends DataFrame object

CPU times: user 149 ms, sys: 75.1 ms, total: 224 ms
Wall time: 231 ms


In [34]:
h5

File(filename=../../assets/hdf_store/data_ts.h5ts, title=np.str_(''), mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) np.str_('')
/data (Group/Timeseries) np.str_('')
/data/y2021 (Group) np.str_('')
/data/y2024 (Group) np.str_('')
/data/y2024/m09 (Group) np.str_('')
/data/y2024/m09/d20 (Group) np.str_('')
/data/y2024/m09/d20/ts_data (Table(np.int64(0),)) np.str_('')
  description := {
  "timestamp": Int64Col(shape=(), dflt=np.int64(0), pos=0),
  "No0": Float64Col(shape=(), dflt=np.float64(0.0), pos=1),
  "No1": Float64Col(shape=(), dflt=np.float64(0.0), pos=2),
  "No2": Float64Col(shape=(), dflt=np.float64(0.0), pos=3),
  "No3": Float64Col(shape=(), dflt=np.float64(0.0), pos=4),
  "No4": Float64Col(shape=(), dflt=np.float64(0.0), pos=5)}
  byteorder := 'little'
  chunkshape := (np.int64(1365),)
/data/y2021/m01 (Group) np.str_('')
/data/y2021/m01/d01 (Group) np.str_('')
/data/y2021/m01/d

In [35]:
# reading subsets of the data from the TsTables table
import datetime

In [36]:
start = datetime.datetime(2021, 1, 2) # defining start and end date for the subset
end = datetime.datetime(2021, 1, 3)

In [37]:
%time subset = ts.read_range(start, end) # read_range takes start and end date as input

CPU times: user 9.27 ms, sys: 6.09 ms, total: 15.4 ms
Wall time: 16 ms


In [38]:
start = datetime.datetime(2021, 1, 2, 12, 30, 0)
end = datetime.datetime(2021, 1, 5, 17, 15, 30)

In [39]:
%time subset = ts.read_range(start, end)

CPU times: user 17.7 ms, sys: 7.44 ms, total: 25.1 ms
Wall time: 25.7 ms


In [40]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 276331 entries, 2021-01-02 12:30:00 to 2021-01-05 17:15:30
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   No0     276331 non-null  float64
 1   No1     276331 non-null  float64
 2   No2     276331 non-null  float64
 3   No3     276331 non-null  float64
 4   No4     276331 non-null  float64
dtypes: float64(5)
memory usage: 12.6 MB


In [41]:
h5.close()

In [42]:
rm ../../assets/hdf_store/data_ts.* # deleting file

### Storing Data with SQLite3

Writing financial time series data from a DataFrame object to a relational database, SQLite3 in this example.

In [16]:
%time data = generate_sample_data(1e6, 5, '1min').round(4)

CPU times: user 220 ms, sys: 44.4 ms, total: 265 ms
Wall time: 351 ms


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000000 entries, 2021-01-01 00:00:00 to 2022-11-26 10:39:00
Freq: min
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   No0     1000000 non-null  float64
 1   No1     1000000 non-null  float64
 2   No2     1000000 non-null  float64
 3   No3     1000000 non-null  float64
 4   No4     1000000 non-null  float64
dtypes: float64(5)
memory usage: 45.8 MB


In [18]:
import sqlite3 as sq3

In [19]:
con = sq3.connect('../../assets/data_sql/data.sql') # opening a connection to a new database file

In [20]:
%time data.to_sql('data', con) # writing the data to the relational database

CPU times: user 3.08 s, sys: 216 ms, total: 3.3 s
Wall time: 3.46 s


1000000

In [21]:
ls -n ../../assets/data_sql/data.*

-rw-r--r--  1 501  20  105316352 21 Sep 09:28 ../../assets/data_sql/data.sql


In [22]:
query = 'SELECT * FROM data WHERE No1 > 103 and No2 < 105' # sql query as a python string object

In [23]:
%time res = con.execute(query).fetchall() # executing query, retrieving all result rows

CPU times: user 90 ms, sys: 27.8 ms, total: 118 ms
Wall time: 118 ms


In [24]:
res[:5] # first five rows of results list object

[('2021-01-02 06:53:00', 98.1084, 103.0133, 98.754, 102.0656, 100.7058),
 ('2021-01-02 07:06:00', 97.7451, 103.0194, 98.986, 101.8655, 100.7693),
 ('2021-01-02 07:08:00', 97.7815, 103.0492, 99.0187, 101.7314, 100.7489),
 ('2021-01-02 07:11:00', 97.846, 103.0989, 98.9572, 101.6346, 100.8165),
 ('2021-01-02 07:12:00', 97.7544, 103.1154, 98.9915, 101.5826, 100.7981)]

In [25]:
len(res)

4402

In [26]:
con.close()

In [27]:
rm ../../assets/data_sql/data.*